In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys

from ng_utilities import *
from ng_full_utilities_test import *

sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *
from registration_utilities import *
from annotation_utilities import *
from metadata import *
from data_manager import *

import matplotlib.pyplot as plt
%matplotlib inline

import neuroglancer
import cv2
import json

with open('struct_to_color.json', 'r') as json_file:
    structure_to_color = json.load( json_file )
    
color_segments=[]
for i in range(1,50):
    color_segments.append(i)

ENABLE_UPLOAD_S3 is not set, default to False.
ENABLE_DOWNLOAD_S3 is not set, default to False.


Setting environment for Precision WorkStation for Alex Newberry
{'MD589': 0.46, 'MD585': 0.46, 'UCSD001': 0.325, 'MD594': 0.46, 'DK17': 0.325}
/media/alexn/BstemAtlasDataBackup/ucsd_brain/CSHL_data_processed/MD585/MD585_cropbox.ini
/media/alexn/BstemAtlasDataBackup/ucsd_brain/CSHL_data_processed/MD585/MD585_cropbox.ini
/media/alexn/BstemAtlasDataBackup/ucsd_brain/CSHL_data_processed/MD589/MD589_cropbox.ini
/media/alexn/BstemAtlasDataBackup/ucsd_brain/CSHL_data_processed/MD589/MD589_cropbox.ini
/media/alexn/BstemAtlasDataBackup/ucsd_brain/CSHL_data_processed/MD594/MD594_cropbox.ini
/media/alexn/BstemAtlasDataBackup/ucsd_brain/CSHL_data_processed/MD594/MD594_cropbox.ini
/media/alexn/BstemAtlasDataBackup/ucsd_brain/CSHL_data_processed/UCSD001/UCSD001_cropbox.ini
/media/alexn/BstemAtlasDataBackup/ucsd_brain/CSHL_data_processed/UCSD001/UCSD001_cropbox.ini


Seems you are using operation INIs to provide cropbox.
Seems you are using operation INIs to provide cropbox.
Seems you are using operation INIs to provide cropbox.
Seems you are using operation INIs to provide cropbox.
No anchor.txt is found. Seems we are using the operation ini to provide anchor. Try to load operation ini.
Failed to cache MD590 anchor: ini file /media/alexn/BstemAtlasDataBackup/ucsd_brain/CSHL_data_processed/MD590/operation_configs/from_none_to_aligned.ini does not exist.
File does not exist: /media/alexn/BstemAtlasDataBackup/ucsd_brain/CSHL_data_processed/MD590/MD590_sorted_filenames.txt
Failed to cache MD590 sections_to_filenames: 
File does not exist: /media/alexn/BstemAtlasDataBackup/ucsd_brain/CSHL_data_processed/MD590/MD590_sorted_filenames.txt
Failed to cache MD590 filenames_to_sections: 
No anchor.txt is found. Seems we are using the operation ini to provide anchor. Try to load operation ini.
Failed to cache MD590 section_limits: ini file /media/alexn/BstemAt

In [9]:
stack = 'MD585'

detector_id = 19
# detector_id = 799

In [2]:
stack = 'MD589'

detector_id = 19
# detector_id = 799

In [2]:
stack = 'MD594'

detector_id = 19
# detector_id = 799

# Neuroglancer code

In [3]:
#Replace: viewer = neuroglancer.Viewer()

#With: 
neuroglancer.set_server_bind_address('0.0.0.0')
viewer = neuroglancer.Viewer()
print(viewer)

ip_name = !hostname -I
print('http://' + ip_name[0].strip() + ':' + viewer.get_viewer_url().split(':')[2]) ##Remote URL

ip_name_str = str(ip_name).replace('\'','').replace('\"','').replace('[','').replace(']','')
real_ip = ip_name_str.split(' ')[0]
print('http://' + real_ip + ':' + viewer.get_viewer_url().split(':')[2]) ##Remote URL

viewer # port 41989, IP 132.239.73.85

http://yuncong-Precision-WorkStation-T7500:46835/v/1677e8fb31d70245f219d2334a443be476cb6bcc/
http://132.239.73.85 172.17.0.1:46835/v/1677e8fb31d70245f219d2334a443be476cb6bcc/
http://132.239.73.85:46835/v/1677e8fb31d70245f219d2334a443be476cb6bcc/


http://yuncong-Precision-WorkStation-T7500:46835/v/1677e8fb31d70245f219d2334a443be476cb6bcc/

In [4]:
# Sets 'Image' layer to be MD585 prep2 images from S3
with viewer.txn() as s:
    s.layers['image'] = neuroglancer.ImageLayer(source='precomputed://https://mousebrainatlas-datajoint-jp2k.s3.amazonaws.com/precomputed/'+stack+'_fullres')
    
    # Resets X/Y/Z plane orientation
    s.navigation.pose.orientation = [0,0,0,1]
    # Zooms out 
#     s.navigation.zoomFactor = 10000 # If 4panel
    s.navigation.zoomFactor = 5000 # If xy
    
    # Resets 3D Viewer Orientation
    s.perspectiveOrientation = [0,0,0,1]
    # Zooms out
    s.perspectiveZoom = 75000
    
    s.layout = 'xy' # '3d'/'4panel'/'xy'
    
# with viewer.txn() as s:
#     print s.layers['navigation']
    
#neuroglancer.stop()

In [21]:
neuroglancer.stop()

# Using Aligned Volumes

In [13]:
! aws s3 ls s3://mousebrainatlas-data/CSHL_volumes/atlasV7/atlasV7_10.0um_scoreVolume/score_volumes/

2018-06-28 18:46:39    1382718 0_test.bp
2018-06-28 18:46:42    4850756 0_test.npz
2018-07-09 20:24:33    1380857 atlasV7_10.0um_scoreVolume_10N_L.bp
2018-07-09 20:24:34         76 atlasV7_10.0um_scoreVolume_10N_L_origin_wrt_canonicalAtlasSpace.txt
2018-07-09 20:24:35    1183865 atlasV7_10.0um_scoreVolume_10N_L_surround_200um.bp
2018-07-09 20:24:36         77 atlasV7_10.0um_scoreVolume_10N_L_surround_200um_origin_wrt_canonicalAtlasSpace.txt
2018-07-09 20:24:37    1379506 atlasV7_10.0um_scoreVolume_10N_R.bp
2018-07-09 20:24:38         75 atlasV7_10.0um_scoreVolume_10N_R_origin_wrt_canonicalAtlasSpace.txt
2018-07-09 20:24:39    1182168 atlasV7_10.0um_scoreVolume_10N_R_surround_200um.bp
2018-07-09 20:24:40         77 atlasV7_10.0um_scoreVolume_10N_R_surround_200um_origin_wrt_canonicalAtlasSpace.txt
2018-07-09 20:25:12    3565927 atlasV7_10.0um_scoreVolume_12N.bp
2018-07-09 20:25:13         76 atlasV7_10.0um_scoreVolume_12N_origin_wrt_canonicalAtlasSpace.txt
2018-07-09 20:25:14    2672104 

2018-07-09 20:23:27    2013971 atlasV7_10.0um_scoreVolume_SNC_R_surround_200um.bp
2018-07-09 20:23:28         77 atlasV7_10.0um_scoreVolume_SNC_R_surround_200um_origin_wrt_canonicalAtlasSpace.txt
2018-07-09 20:23:29    8753157 atlasV7_10.0um_scoreVolume_SNR_L.bp
2018-07-09 20:23:30         78 atlasV7_10.0um_scoreVolume_SNR_L_origin_wrt_canonicalAtlasSpace.txt
2018-07-09 20:23:31    5845521 atlasV7_10.0um_scoreVolume_SNR_L_surround_200um.bp
2018-07-09 20:23:33         78 atlasV7_10.0um_scoreVolume_SNR_L_surround_200um_origin_wrt_canonicalAtlasSpace.txt
2018-07-09 20:23:34    8759638 atlasV7_10.0um_scoreVolume_SNR_R.bp
2018-07-09 20:23:35         77 atlasV7_10.0um_scoreVolume_SNR_R_origin_wrt_canonicalAtlasSpace.txt
2018-07-09 20:23:36    5847160 atlasV7_10.0um_scoreVolume_SNR_R_surround_200um.bp
2018-07-09 20:23:38         77 atlasV7_10.0um_scoreVolume_SNR_R_surround_200um_origin_wrt_canonicalAtlasSpace.txt
2018-07-09 20:24:15   13486557 atlasV7_10.0um_scoreVolume_Sp5C_L.bp
2018-07-09 2

In [9]:
! aws s3 ls s3://mousebrainatlas-data/CSHL_volumes/MD589/

                           PRE MD589_10.0um_annotationAsScoreVolume/
                           PRE MD589_detector19_10.0um_scoreVolume/
                           PRE MD589_detector998_10.0um_scoreVolume/
                           PRE MD589_down32_annotationAsScoreVolume/
                           PRE MD589_down32_annotationAsScoreVolume_warp8_MD589_down32_annotationAsScoreVolume/
                           PRE MD589_down32_annotationAsScoreVolume_warp9_MD589_down32_annotationAsScoreVolume/
                           PRE MD589_down32_annotationVolume/
                           PRE MD589_down32_annotationVolume_warp8_MD589_down32_annotationVolume/
                           PRE MD589_down32_intensityVolume/
                           PRE MD589_prep2_detector15_down32_scoreVolume/
                           PRE MD589_prep2_detector1_down32_scoreVolume/
                           PRE MD589_prep2_down32_annotationAsScoreVolume/
                           PRE MD589_prep4_down32_intensit

In [6]:
structure = '12N'
# color_codes{'blue:'1,'red',2,'yellow':3}

str_contour, first_sec, last_sec = image_contour_generator( stack, detector_id, structure, use_local_alignment=True, image_prep=2, threshold=0.2)

ng_structure_volume_normal = add_structure_to_neuroglancer( viewer, str_contour, structure, stack, first_sec, last_sec, \
                                                    color_radius=5, xy_ng_resolution_um=10, threshold=0.2, color=5, \
                                                    solid_volume=False, no_offset_big_volume=False, save_results=False, \
                                                    return_with_offsets=False, add_to_ng=True, human_annotation=False)

plt.imshow(ng_structure_volume_normal[20,:,:])

IOError: [Errno 2] No such file or directory: u'/media/alexn/BstemAtlasDataBackup/ucsd_brain/CSHL_volumes/atlasV7/atlasV7_10.0um_scoreVolume_12N_warp7_MD589_detector19_10.0um_scoreVolume_12N_10.0um/score_volumes/atlasV7_10.0um_scoreVolume_12N_warp7_MD589_detector19_10.0um_scoreVolume_12N_10.0um_12N.bp'

In [12]:
for structure in all_structures_total:
    str_contour, first_sec, last_sec = image_contour_generator( stack, detector_id, structure, use_local_alignment=True, image_prep=2, threshold=0.2)

    add_structure_to_neuroglancer( viewer, str_contour, structure, stack, first_sec, last_sec, \
                                  color_radius=5, xy_ng_resolution_um=10, threshold=0.2, color=1, \
                                  solid_volume=False, no_offset_big_volume=False, save_results=False,\
                                  return_with_offsets=False, add_to_ng=True, human_annotation=False)

/media/alexn/BstemAtlasDataBackup/ucsd_brain/CSHL_data_processed/MD585/MD585_cropbox.ini


Seems you are using operation INIs to provide cropbox.
2 contours of reconstructed volume is found at position 10 ([6, 3]). Use the longest one.
2 contours of reconstructed volume is found at position 52 ([2, 1]). Use the longest one.
contour has less than three vertices. Ignore.


Number of valid sections:
21
First valid section: 188
Last valid section: 208
num_valid_sections: 21



(20, 30, 58)
/media/alexn/BstemAtlasDataBackup/ucsd_brain/CSHL_data_processed/MD585/MD585_cropbox.ini


Seems you are using operation INIs to provide cropbox.


Number of valid sections:
44
First valid section: 180
Last valid section: 223
num_valid_sections: 44



(43, 62, 114)
/media/alexn/BstemAtlasDataBackup/ucsd_brain/CSHL_data_processed/MD585/MD585_cropbox.ini


Seems you are using operation INIs to provide cropbox.


KeyboardInterrupt: 

In [65]:
# CRAETE ENTIRE BRAIN VOLUME
xy_ng_resolution_um = 5

with open('struct_reverse_2.json', 'r') as json_file:
    structure_to_color = json.load( json_file )

    
# MD585: x_um = 35617,           y_um = 26086
# MD585: x_pixels_.46res = x_um*0.46,  y_pixels_.46res = y_um*0.46
# MD585: x_pixels_newres = x_pixels_.46res*(0.46/newres), y_pixels_newres = y_pixels_.46res*(0.46/newres)
# microns/resolution
y_voxels = int( 26086*0.46*(.46/xy_ng_resolution_um) + 0.5)
x_voxels = int( 35617*0.46*(.46/xy_ng_resolution_um) + 0.5)
full_brain_volumes = np.zeros((268,y_voxels,x_voxels), dtype=np.uint8)

for structure in all_structures_total:
    str_contour, first_sec, last_sec = image_contour_generator( stack, detector_id, structure, use_local_alignment=True, image_prep=2, threshold=0.5)
    
    try:
        color=structure_to_color[structure]
    except:
        color=2
    
    str_volume, xyz_offsets = add_structure_to_neuroglancer( viewer, str_contour, structure, stack, first_sec, last_sec, \
                                          color_radius=5, xy_ng_resolution_um=xy_ng_resolution_um, threshold=0.5, color=color, \
                                          solid_volume=False, no_offset_big_volume=True, save_results=False, return_with_offsets=True, \
                                          add_to_ng=False, human_annotation=False )
    
    z_len, y_len, x_len = np.shape(str_volume)
    full_brain_volumes[0:z_len, 0:y_len, 0:x_len] += str_volume



color_segments=[]
for i in range(1,50):
    color_segments.append(i)

with viewer.txn() as s:
    #s.layers[ display_name ] = neuroglancer.SegmentationLayer(
    s.layers[ "full_brain" ] = neuroglancer.SegmentationLayer(
        source = neuroglancer.LocalVolume(
            data=full_brain_volumes, # Z,Y,X
            voxel_size=[ xy_ng_resolution_um*1000, xy_ng_resolution_um*1000,20000], # X Y Z
            voxel_offset = [ 0, 0, 0] # X Y Z
        ),
        segments = color_segments
    )

/media/alexn/BstemAtlasDataBackup/ucsd_brain/CSHL_data_processed/MD585/MD585_cropbox.ini


Seems you are using operation INIs to provide cropbox.
2 contours of reconstructed volume is found at position 48 ([2, 1]). Use the longest one.
contour has less than three vertices. Ignore.


Number of valid sections:
17
First valid section: 190
Last valid section: 206
num_valid_sections: 17



(16, 49, 101)
/media/alexn/BstemAtlasDataBackup/ucsd_brain/CSHL_data_processed/MD585/MD585_cropbox.ini


Seems you are using operation INIs to provide cropbox.
contour has less than three vertices. Ignore.
2 contours of reconstructed volume is found at position 96 ([16, 1]). Use the longest one.


Number of valid sections:
40
First valid section: 182
Last valid section: 221
num_valid_sections: 40



(39, 115, 205)
/media/alexn/BstemAtlasDataBackup/ucsd_brain/CSHL_data_processed/MD585/MD585_cropbox.ini


...

Number of valid sections:
27
First valid section: 84
Last valid section: 110
num_valid_sections: 27



(26, 131, 177)
/media/alexn/BstemAtlasDataBackup/ucsd_brain/CSHL_data_processed/MD585/MD585_cropbox.ini


Seems you are using operation INIs to provide cropbox.
contour has less than three vertices. Ignore.
2 contours of reconstructed volume is found at position 47 ([1, 1]). Use the longest one.
contour has less than three vertices. Ignore.


Number of valid sections:
52
First valid section: 285
Last valid section: 336
num_valid_sections: 52



(51, 117, 181)
/media/alexn/BstemAtlasDataBackup/ucsd_brain/CSHL_data_processed/MD585/MD585_cropbox.ini


Seems you are using operation INIs to provide cropbox.
2 contours of reconstructed volume is found at position 24 ([47, 1]). Use the longest one.
2 contours of reconstructed volume is found at position 110 ([3, 1]). Use the longest one.
2 contours of reconstructed volume is found at position 18 ([3, 2]). Use the longest one.
2 contours of reconstructed volume is found at position 102 ([3, 2]). Use the longest one.


Number of valid sections:
49
First valid section: 86
Last valid section: 134
num_valid_sections: 49



(48, 127, 193)


In [22]:
color_array = np.ones((1,80,80), dtype=np.uint8)
color_array[0,0:49,0]= np.array(color_segments)
color_array[0,0,0:49]= np.array(color_segments)


with viewer.txn() as s:
    #s.layers[ display_name ] = neuroglancer.SegmentationLayer(
    s.layers[ "color_test" ] = neuroglancer.SegmentationLayer(
        source = neuroglancer.LocalVolume(
            data= color_array, # Z,Y,X
            voxel_size=[ 50000, 50000,200000], # X Y Z
            voxel_offset = [ 0, 0, 0] # X Y Z
        ),
        segments = color_segments
    )

# Human Annotation Contours

In [16]:
! ls $ROOT_DIR/CSHL_labelings_v3/MD585/MD585_annotation_contours_05302018161849.hdf

MD585_annotation_contours_05302018161849.hdf
MD585_annotation_grid_indices.h5
MD585_annotation_v3.h5
MD585_annotation_win1_05302018161849_grid_indices_lookup.hdf
MD585_annotation_win12_05302018161849_grid_indices_lookup.hdf
MD585_annotation_win7_08012017212649_grid_indices_lookup.hdf


In [15]:
! ls $ROOT_DIR/CSHL_labelings_v3/$stack/

MD589_annotation_contours_05302018164231.hdf
MD589_annotation_grid_indices.h5
MD589_annotation_structures_10312017220057.hdf
MD589_annotation_v3_09252016160841.h5
MD589_annotation_v3_09252016172320.h5
MD589_annotation_v3_09252016195300.h5
MD589_annotation_v3.h5
MD589_annotation_win7_05302018164231_grid_indices_lookup.hdf


In [10]:
! ls $ROOT_DIR/CSHL_labelings_v3/MD585/

MD585_annotation_contours_05302018161849.hdf
MD585_annotation_grid_indices.h5
MD585_annotation_v3.h5
MD585_annotation_win1_05302018161849_grid_indices_lookup.hdf
MD585_annotation_win12_05302018161849_grid_indices_lookup.hdf
MD585_annotation_win2_05302018161849_grid_indices_lookup.hdf
MD585_annotation_win7_08012017212649_grid_indices_lookup.hdf


In [11]:
! ls $ROOT_DIR/CSHL_labelings_v3/MD589/

MD589_annotation_contours_05302018164231.hdf
MD589_annotation_grid_indices.h5
MD589_annotation_structures_10312017220057.hdf
MD589_annotation_v3_09252016160841.h5
MD589_annotation_v3_09252016172320.h5
MD589_annotation_v3_09252016195300.h5
MD589_annotation_v3.h5
MD589_annotation_win7_05302018164231_grid_indices_lookup.hdf


In [7]:
! ls $ROOT_DIR/CSHL_labelings_v3/MD594/

MD594_annotation_contours_05312018035134.hdf
MD594_annotation_structures_11012017013215.hdf
MD594_annotation_structuresHanddrawn_05312018043152.hdf


In [9]:
hand_annotations

,edits,name,origin,resolution,side,volume
109066512106773,[],RMC,"[1300, 526, 583]",8.0um,R,blpk�� ��        \n       JSON     D...
113058115045632,[],LC,"[1507, 466, 648]",8.0um,R,blpk�� ��        \n       JSON     E...
116837006160528,[],LC,"[1509, 459, 448]",8.0um,L,blpk�� ��        \n       JSON     E...
130558812337002,[],Sp5O,"[1579, 503, 748]",8.0um,R,blpk   `l        (       JSON     F ...
144876136029104,[],LRt,"[1799, 630, 398]",8.0um,L,blpk   @?               JSON     E ...
144979757486365,[],5N,"[1510, 536, 368]",8.0um,L,blpk؝ ؝        \n       JSON     D  ...
146768542392515,[],VLL,"[1427, 495, 323]",8.0um,L,blpk    ;                JSON     F ...
147231063283877,[],DC,"[1563, 437, 758]",8.0um,R,blpk   �               JSON     F ...
156871905186678,[],SNR,"[1251, 525, 633]",8.0um,R,blpk   ��        (       JSON     G ...
159287725634090,[],VLL,"[1399, 507, 711]",8.0um,R,blpk   ��                JSON     F ...


In [5]:

# MD585
if stack=="MD585":
    hand_annotations = load_hdf_v2(ROOT_DIR+'/CSHL_labelings_v3/MD585/MD585_annotation_contours_05302018161849.hdf')
if stack=="MD589":
    hand_annotations = load_hdf_v2(ROOT_DIR+'/CSHL_labelings_v3/MD589/MD589_annotation_contours_05302018164231.hdf')
if stack=="MD594":
    hand_annotations = load_hdf_v2(ROOT_DIR+'/CSHL_labelings_v3/MD594/MD594_annotation_contours_05312018035134.hdf')
    
# Converts thw weird coordinate frame to a normal-ass prep2 raw frame
hand_annotations = convert_annotation_v3_original_to_aligned_cropped_v2(\
                                        hand_annotations, stack=stack,\
                                        out_resolution='raw',
                                        prep_id=2)


num_annotations = len(hand_annotations)
#hand_annotations.keys()


MD585_ng_section_min = 83
MD585_ng_section_max = 536
# For sections before the 22nd section there is a weird offset that this compensates for
MD585_abberation_correction = [9872, 4258] # UNUSED Currently

def get_dense_coordinates( coor_list ):
    dense_coor_list = []
    # Shortest distance, x, y

    #for x, y in coor_list:
    for i in range(len(coor_list)-1):
        x, y = coor_list[i]
        x_next, y_next = coor_list[i+1]
        
        x_mid = (x+x_next)/2
        y_mid = (y+y_next)/2
        
        
        dense_coor_list.append([x,y])
        dense_coor_list.append( [x_mid, y_mid] )
        
        if i==len(coor_list)-2:
            dense_coor_list.append( [x_next, y_next] )
            x, y = coor_list[0]
            x_mid = (x+x_next)/2
            y_mid = (y+y_next)/2
            dense_coor_list.append( [x_mid, y_mid] )
        
    return dense_coor_list

def get_contours_from_annotations( stack, target_str, densify=0 ):
    str_contours_annotation = {}

    for i in range(num_annotations):
        structure = hand_annotations['name'][i]
        side = hand_annotations['side'][i]
        section = hand_annotations['section'][i]
        
        if side=='R' or side=='L':
            structure = structure+'_'+side
            
        if structure==target_str:
            vertices = hand_annotations['vertices'][i]
            
            for i in range(densify):
                vertices = get_dense_coordinates( vertices )
            
            # Skip sections before the 22nd prep2 section for MD585 as there are clear errors
            if stack=='MD585' and section < MD585_ng_section_min+22:
                #vertices = vertices - np.array(MD585_abberation_correction)
                continue

            str_contours_annotation[section] = {}
            str_contours_annotation[section][structure] = {}
            str_contours_annotation[section][structure][1] = vertices

    first_sec = np.min(str_contours_annotation.keys())
    last_sec = np.max(str_contours_annotation.keys())
    
    return str_contours_annotation, first_sec, last_sec

/media/alexn/BstemAtlasDataBackup/ucsd_brain/CSHL_data_processed/MD594/MD594_cropbox.ini


Seems you are using operation INIs to provide cropbox.


In [22]:
# for target_str in all_structures_total[0:3]:
for target_str in ['12N']:
    str_contours_annotation, first_sec, last_sec = get_contours_from_annotations( stack, target_str, densify=4 )

    ng_structure_volume = add_structure_to_neuroglancer( viewer, str_contours_annotation, target_str, stack, first_sec, last_sec, \
                                                    color_radius=3, xy_ng_resolution_um=10, threshold=1, color=4, \
                                                    solid_volume=False, no_offset_big_volume=False, save_results=False, \
                                                    return_with_offsets=False, add_to_ng=True, human_annotation=True )

[206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245]
(39, 72, 114)


In [25]:
# for target_str in all_structures_total[0:3]:
for target_str in ['7n_R']:
    str_contours_annotation, first_sec, last_sec = get_contours_from_annotations( stack, target_str, densify=4 )

    ng_structure_volume = add_structure_to_neuroglancer( viewer, str_contours_annotation, target_str, stack, first_sec, last_sec, \
                                                    color_radius=3, xy_ng_resolution_um=10, threshold=1, color=4, \
                                                    solid_volume=False, no_offset_big_volume=False, save_results=False, \
                                                    return_with_offsets=False, add_to_ng=True, human_annotation=True )

[256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311, 312, 313, 314, 315, 316, 317, 318, 250, 251, 252, 253, 254, 255]
(68, 109, 90)


In [11]:
# for target_str in all_structures_total[0:3]:
for target_str in ['7n_R']:
    str_contours_annotation, first_sec, last_sec = get_contours_from_annotations( stack, target_str, densify=4 )

    ng_structure_volume = add_structure_to_neuroglancer( viewer, str_contours_annotation, target_str, stack, first_sec, last_sec, \
                                                    color_radius=3, xy_ng_resolution_um=10, threshold=1, color=4, \
                                                    solid_volume=False, no_offset_big_volume=True, save_results=False, \
                                                    return_with_offsets=False, add_to_ng=True, human_annotation=True )

(68, 109, 90)


In [ ]:
# REAL
# 16384 x 12000 pixels raw
# 35617 x 26086 um raw

In [23]:
x_voxels

755

In [6]:
# Add all annotated brains to the viewer
xy_ng_resolution_um = 5
color_radius = 3

# MD585: x_um = 35617,           y_um = 26086
# MD585: x_pixels_.46res = x_um*0.46,  y_pixels_.46res = y_um*0.46
# MD585: x_pixels_newres = x_pixels_.46res*(0.46/newres), y_pixels_newres = y_pixels_.46res*(0.46/newres)
# microns/resolution
y_voxels = 1+int( 26086*0.46*(.46/xy_ng_resolution_um) + 0.5)
x_voxels = 1+int( 35617*0.46*(.46/xy_ng_resolution_um) + 0.5)
full_brain_volume_annotated = np.zeros((268,y_voxels,x_voxels), dtype=np.uint8)

for target_str in all_structures_total:
# for target_str in['VCA_L','7n_R','7n_L']:
    print(target_str)
    str_contours_annotation, first_sec, last_sec = get_contours_from_annotations( stack, target_str, densify=4 )
    
    try:
        color=structure_to_color[target_str]
    except Exception as e:
        print(e)
        color=4
    
    str_volume, xyz_str_offsets = add_structure_to_neuroglancer( \
                            viewer, str_contours_annotation, target_str, stack, first_sec, last_sec, \
                            color_radius=color_radius, xy_ng_resolution_um=xy_ng_resolution_um, threshold=1, color=color, \
                            solid_volume=False, no_offset_big_volume=True, save_results=True, \
                            return_with_offsets=True, add_to_ng=False, human_annotation=True  )
    
    z_len, y_len, x_len = np.shape(str_volume)
#     full_brain_volume_annotated[0:z_len, 0:y_len, 0:x_len] = str_volume.copy()
    for z in range( xyz_str_offsets[2], z_len ):
        for y in range( xyz_str_offsets[1], y_len ):
            for x in range( xyz_str_offsets[0], x_len ):
                structure_val = str_volume[z, y, x]
                if structure_val==0:
                    continue
                else:
                    try:
                        full_brain_volume_annotated[z, y, x] = structure_val
                    except Exception as e:
                        print(e)
    
with viewer.txn() as s:
    s.layers[ stack+"_Atlas" ] = neuroglancer.SegmentationLayer(
        source = neuroglancer.LocalVolume(
            data=full_brain_volume_annotated, # Z,Y,X
            voxel_size=[ xy_ng_resolution_um*1000, xy_ng_resolution_um*1000,20000], # X Y Z
            voxel_offset = [ 0, 0, 0] # X Y Z
        ),
        segments = color_segments
    )

AP
(19, 56, 138)
12N
(39, 116, 262)
RtTg
'RtTg'
(82, 216, 243)
SC
(172, 395, 403)
IC
(171, 367, 326)
5N_R
(22, 153, 132)
5N_L
(24, 156, 145)
6N_R
(12, 40, 50)
6N_L
(13, 48, 41)
7N_R
(40, 148, 171)
7N_L
(44, 147, 196)
7n_R
'7n_R'
(62, 234, 173)
7n_L
'7n_L'
(65, 248, 154)
Amb_R
'Amb_R'
(5, 44, 100)
Amb_L
'Amb_L'
(5, 38, 106)
LC_R
(12, 115, 161)
LC_L
(12, 121, 161)
LRt_R
'LRt_R'
(37, 164, 290)
LRt_L
'LRt_L'
(35, 144, 292)
Pn_R
'Pn_R'
(43, 155, 212)
Pn_L
'Pn_L'
(52, 159, 220)
Tz_R
'Tz_R'
(24, 59, 187)
Tz_L
'Tz_L'
(20, 61, 182)
VLL_R
(30, 371, 150)
VLL_L
(35, 386, 160)
RMC_R
(25, 137, 141)
RMC_L
(26, 139, 116)
SNC_R
(48, 227, 98)
SNC_L
(45, 220, 88)
SNR_R
(62, 314, 211)
SNR_L
(61, 314, 194)
3N_R
(11, 84, 117)
3N_L
(10, 75, 121)
4N_R
(5, 47, 39)
4N_L
(7, 49, 38)
Sp5I_R
'Sp5I_R'
(36, 261, 223)
Sp5I_L
'Sp5I_L'
(34, 236, 205)
Sp5O_R
'Sp5O_R'
(33, 267, 375)
Sp5O_L
'Sp5O_L'
(34, 240, 351)
Sp5C_R
'Sp5C_R'
(52, 300, 331)
Sp5C_L
'Sp5C_L'
(67, 286, 392)
index 1508 is out of bounds for axis 2 with siz

index 1508 is out of bounds for axis 2 with size 1508
index 1508 is out of bounds for axis 2 with size 1508
index 1509 is out of bounds for axis 2 with size 1508
index 1508 is out of bounds for axis 2 with size 1508
index 1509 is out of bounds for axis 2 with size 1508
index 1510 is out of bounds for axis 2 with size 1508
index 1508 is out of bounds for axis 2 with size 1508
index 1509 is out of bounds for axis 2 with size 1508
index 1510 is out of bounds for axis 2 with size 1508
index 1508 is out of bounds for axis 2 with size 1508
index 1509 is out of bounds for axis 2 with size 1508
index 1510 is out of bounds for axis 2 with size 1508
index 1511 is out of bounds for axis 2 with size 1508
index 1508 is out of bounds for axis 2 with size 1508
index 1509 is out of bounds for axis 2 with size 1508
index 1510 is out of bounds for axis 2 with size 1508
index 1511 is out of bounds for axis 2 with size 1508
index 1512 is out of bounds for axis 2 with size 1508
index 1508 is out of bounds 

index 1509 is out of bounds for axis 2 with size 1508
index 1508 is out of bounds for axis 2 with size 1508
index 1508 is out of bounds for axis 2 with size 1508
index 1509 is out of bounds for axis 2 with size 1508
index 1508 is out of bounds for axis 2 with size 1508
index 1509 is out of bounds for axis 2 with size 1508
index 1510 is out of bounds for axis 2 with size 1508
index 1508 is out of bounds for axis 2 with size 1508
index 1509 is out of bounds for axis 2 with size 1508
index 1510 is out of bounds for axis 2 with size 1508
index 1511 is out of bounds for axis 2 with size 1508
index 1512 is out of bounds for axis 2 with size 1508
index 1508 is out of bounds for axis 2 with size 1508
index 1509 is out of bounds for axis 2 with size 1508
index 1510 is out of bounds for axis 2 with size 1508
index 1511 is out of bounds for axis 2 with size 1508
index 1512 is out of bounds for axis 2 with size 1508
index 1513 is out of bounds for axis 2 with size 1508
index 1508 is out of bounds 

index 1509 is out of bounds for axis 2 with size 1508
index 1510 is out of bounds for axis 2 with size 1508
index 1511 is out of bounds for axis 2 with size 1508
index 1512 is out of bounds for axis 2 with size 1508
index 1513 is out of bounds for axis 2 with size 1508
index 1514 is out of bounds for axis 2 with size 1508
index 1515 is out of bounds for axis 2 with size 1508
index 1516 is out of bounds for axis 2 with size 1508
index 1517 is out of bounds for axis 2 with size 1508
index 1518 is out of bounds for axis 2 with size 1508
index 1519 is out of bounds for axis 2 with size 1508
index 1520 is out of bounds for axis 2 with size 1508
index 1521 is out of bounds for axis 2 with size 1508
index 1522 is out of bounds for axis 2 with size 1508
index 1523 is out of bounds for axis 2 with size 1508
index 1524 is out of bounds for axis 2 with size 1508
index 1525 is out of bounds for axis 2 with size 1508
index 1508 is out of bounds for axis 2 with size 1508
index 1509 is out of bounds 

index 1540 is out of bounds for axis 2 with size 1508
index 1529 is out of bounds for axis 2 with size 1508
index 1530 is out of bounds for axis 2 with size 1508
index 1531 is out of bounds for axis 2 with size 1508
index 1532 is out of bounds for axis 2 with size 1508
index 1533 is out of bounds for axis 2 with size 1508
index 1534 is out of bounds for axis 2 with size 1508
index 1535 is out of bounds for axis 2 with size 1508
index 1536 is out of bounds for axis 2 with size 1508
index 1537 is out of bounds for axis 2 with size 1508
index 1538 is out of bounds for axis 2 with size 1508
index 1539 is out of bounds for axis 2 with size 1508
index 1527 is out of bounds for axis 2 with size 1508
index 1528 is out of bounds for axis 2 with size 1508
index 1529 is out of bounds for axis 2 with size 1508
index 1530 is out of bounds for axis 2 with size 1508
index 1531 is out of bounds for axis 2 with size 1508
index 1532 is out of bounds for axis 2 with size 1508
index 1533 is out of bounds 

...

index 269 is out of bounds for axis 0 with size 268
index 269 is out of bounds for axis 0 with size 268
index 269 is out of bounds for axis 0 with size 268
index 269 is out of bounds for axis 0 with size 268
index 269 is out of bounds for axis 0 with size 268
index 269 is out of bounds for axis 0 with size 268
index 269 is out of bounds for axis 0 with size 268
index 269 is out of bounds for axis 0 with size 268
index 269 is out of bounds for axis 0 with size 268
index 269 is out of bounds for axis 0 with size 268
index 269 is out of bounds for axis 0 with size 268
index 269 is out of bounds for axis 0 with size 268
index 269 is out of bounds for axis 0 with size 268
index 269 is out of bounds for axis 0 with size 268
index 269 is out of bounds for axis 0 with size 268
index 269 is out of bounds for axis 0 with size 268
index 269 is out of bounds for axis 0 with size 268
index 269 is out of bounds for axis 0 with size 268
index 269 is out of bounds for axis 0 with size 268
index 269 is

index 269 is out of bounds for axis 0 with size 268
index 269 is out of bounds for axis 0 with size 268
index 269 is out of bounds for axis 0 with size 268
index 269 is out of bounds for axis 0 with size 268
index 269 is out of bounds for axis 0 with size 268
index 269 is out of bounds for axis 0 with size 268
index 269 is out of bounds for axis 0 with size 268
index 270 is out of bounds for axis 0 with size 268
index 270 is out of bounds for axis 0 with size 268
index 270 is out of bounds for axis 0 with size 268
index 270 is out of bounds for axis 0 with size 268
index 270 is out of bounds for axis 0 with size 268
index 270 is out of bounds for axis 0 with size 268
index 270 is out of bounds for axis 0 with size 268
index 270 is out of bounds for axis 0 with size 268
index 270 is out of bounds for axis 0 with size 268
index 270 is out of bounds for axis 0 with size 268
index 270 is out of bounds for axis 0 with size 268
index 270 is out of bounds for axis 0 with size 268
index 270 is

index 270 is out of bounds for axis 0 with size 268
index 270 is out of bounds for axis 0 with size 268
index 270 is out of bounds for axis 0 with size 268
index 270 is out of bounds for axis 0 with size 268
index 270 is out of bounds for axis 0 with size 268
index 270 is out of bounds for axis 0 with size 268
index 270 is out of bounds for axis 0 with size 268
index 270 is out of bounds for axis 0 with size 268
index 270 is out of bounds for axis 0 with size 268
index 270 is out of bounds for axis 0 with size 268
index 270 is out of bounds for axis 0 with size 268
index 270 is out of bounds for axis 0 with size 268
index 270 is out of bounds for axis 0 with size 268
index 270 is out of bounds for axis 0 with size 268
index 270 is out of bounds for axis 0 with size 268
index 270 is out of bounds for axis 0 with size 268
index 270 is out of bounds for axis 0 with size 268
index 270 is out of bounds for axis 0 with size 268
index 270 is out of bounds for axis 0 with size 268
index 270 is

index 270 is out of bounds for axis 0 with size 268
index 270 is out of bounds for axis 0 with size 268
index 270 is out of bounds for axis 0 with size 268
index 270 is out of bounds for axis 0 with size 268
index 270 is out of bounds for axis 0 with size 268
index 270 is out of bounds for axis 0 with size 268
index 270 is out of bounds for axis 0 with size 268
index 270 is out of bounds for axis 0 with size 268
index 270 is out of bounds for axis 0 with size 268
index 270 is out of bounds for axis 0 with size 268
index 270 is out of bounds for axis 0 with size 268
index 270 is out of bounds for axis 0 with size 268
index 270 is out of bounds for axis 0 with size 268
index 270 is out of bounds for axis 0 with size 268
index 270 is out of bounds for axis 0 with size 268
index 270 is out of bounds for axis 0 with size 268
index 270 is out of bounds for axis 0 with size 268
index 270 is out of bounds for axis 0 with size 268
index 270 is out of bounds for axis 0 with size 268
index 270 is

index 270 is out of bounds for axis 0 with size 268
VCA_L
(30, 224, 249)
VCP_R
(27, 223, 136)
VCP_L
(23, 258, 156)
DC_R
(51, 119, 231)
DC_L
(50, 146, 195)


In [10]:
with viewer.txn() as s:
    s.layers[ stack+"_Atlas" ] = neuroglancer.SegmentationLayer(
        source = neuroglancer.LocalVolume(
            data=full_brain_volume_annotated, # Z,Y,X
            voxel_size=[ xy_ng_resolution_um*1000, xy_ng_resolution_um*1000,20000], # X Y Z
            voxel_offset = [ 0, 0, 0] # X Y Z
        ),
        segments = color_segments
    )

In [7]:
full_annotation_fp = NEUROGLANCER_ROOT+stack+'/human_annotation/solid_volume_'+str(xy_ng_resolution_um)+'um/'
full_annotation_fn = full_annotation_fp+'volume_colored.npy'
print('Saving to :'+full_annotation_fp+full_annotation_fn)

if not os.path.exists( full_annotation_fp ):
    os.makedirs(full_annotation_fp)
    
np.save( full_annotation_fn, full_brain_volume_annotated )
# radius <= 1 : wire
#>radius <= 2 : thin
#>radius <= 3.5 : ''

Saving to :/media/alexn/BstemAtlasDataBackup/neuroglancer_volumes/MD594/human_annotation/solid_volume_5um//media/alexn/BstemAtlasDataBackup/neuroglancer_volumes/MD594/human_annotation/solid_volume_5um/volume_colored.npy


In [50]:
! mkdir /home/alexn/Desktop/neuroglancer_binary_volumes/human_annotations_5um/

# TESTING

In [8]:
! aws s3 rm --recursive s3://test-bucket-sid/alex_neuroglancer_volumes

delete: s3://test-bucket-sid/alex_neuroglancer_volumes/MD585/human_annotation/structure_volumes_10um/10N_L_volume.npy
delete: s3://test-bucket-sid/alex_neuroglancer_volumes/MD585/human_annotation/solid_volume_5um/volume_colored.npy
delete: s3://test-bucket-sid/alex_neuroglancer_volumes/MD585/human_annotation/solid_volume_5um/volume_colored_wire.npy
delete: s3://test-bucket-sid/alex_neuroglancer_volumes/MD585/human_annotation/structure_volumes_10um/3N_R_volume.npy
delete: s3://test-bucket-sid/alex_neuroglancer_volumes/MD585/human_annotation/structure_volumes_10um/10N_R_volume.npy
delete: s3://test-bucket-sid/alex_neuroglancer_volumes/MD585/human_annotation/structure_volumes_10um/4N_L_volume.npy
delete: s3://test-bucket-sid/alex_neuroglancer_volumes/MD585/human_annotation/solid_volume_5um/volume_colored_thin.npy
delete: s3://test-bucket-sid/alex_neuroglancer_volumes/MD585/human_annotation/structure_volumes_10um/3N_L_volume.npy
delete: s3://test-bucket-sid/alex_neuroglancer_volumes/MD585/

delete: s3://test-bucket-sid/alex_neuroglancer_volumes/MD585/human_annotation/structure_volumes_50um/Amb_L_volume.npy
delete: s3://test-bucket-sid/alex_neuroglancer_volumes/MD585/human_annotation/structure_volumes_50um/DC_L_volume.npy
delete: s3://test-bucket-sid/alex_neuroglancer_volumes/MD585/human_annotation/structure_volumes_50um/IC_volume.npy
delete: s3://test-bucket-sid/alex_neuroglancer_volumes/MD585/human_annotation/structure_volumes_50um/DC_R_volume.npy
delete: s3://test-bucket-sid/alex_neuroglancer_volumes/MD585/human_annotation/structure_volumes_50um/LC_L_volume.npy
delete: s3://test-bucket-sid/alex_neuroglancer_volumes/MD585/human_annotation/structure_volumes_50um/LRt_L_volume.npy
delete: s3://test-bucket-sid/alex_neuroglancer_volumes/MD585/human_annotation/structure_volumes_50um/LRt_R_volume.npy
delete: s3://test-bucket-sid/alex_neuroglancer_volumes/MD585/human_annotation/structure_volumes_50um/PBG_L_volume.npy
delete: s3://test-bucket-sid/alex_neuroglancer_volumes/MD585/h

delete: s3://test-bucket-sid/alex_neuroglancer_volumes/MD585/human_annotation/structure_volumes_5um/SNR_R_volume.npy
delete: s3://test-bucket-sid/alex_neuroglancer_volumes/MD585/human_annotation/structure_volumes_5um/Sp5O_L_volume.npy
delete: s3://test-bucket-sid/alex_neuroglancer_volumes/MD585/human_annotation/structure_volumes_5um/Sp5I_R_volume.npy
delete: s3://test-bucket-sid/alex_neuroglancer_volumes/MD585/human_annotation/structure_volumes_5um/Sp5C_L_volume.npy
delete: s3://test-bucket-sid/alex_neuroglancer_volumes/MD585/human_annotation/structure_volumes_5um/Sp5C_R_volume.npy
delete: s3://test-bucket-sid/alex_neuroglancer_volumes/MD585/human_annotation/structure_volumes_5um/Tz_R_volume.npy
delete: s3://test-bucket-sid/alex_neuroglancer_volumes/MD585/human_annotation/structure_volumes_5um/VCA_L_volume.npy
delete: s3://test-bucket-sid/alex_neuroglancer_volumes/MD585/human_annotation/structure_volumes_5um/Tz_L_volume.npy
delete: s3://test-bucket-sid/alex_neuroglancer_volumes/MD585/h

In [9]:
! aws s3 cp --recursive $NEUROGLANCER_ROOT s3://test-bucket-sid/alex_neuroglancer_volumes

upload: ../../../../../media/alexn/BstemAtlasDataBackup/neuroglancer_volumes/MD585/human_annotation/solid_volume_5um/volume_colored.npy to s3://test-bucket-sid/alex_neuroglancer_volumes/MD585/human_annotation/solid_volume_5um/volume_colored.npy
upload: ../../../../../media/alexn/BstemAtlasDataBackup/neuroglancer_volumes/MD585/human_annotation/solid_volume_5um/volume_colored_wire.npy to s3://test-bucket-sid/alex_neuroglancer_volumes/MD585/human_annotation/solid_volume_5um/volume_colored_wire.npy
upload: ../../../../../media/alexn/BstemAtlasDataBackup/neuroglancer_volumes/MD585/human_annotation/solid_volume_5um/volume_colored_thin.npy to s3://test-bucket-sid/alex_neuroglancer_volumes/MD585/human_annotation/solid_volume_5um/volume_colored_thin.npy
upload: ../../../../../media/alexn/BstemAtlasDataBackup/neuroglancer_volumes/MD585/human_annotation/structure_volumes_10um/10N_L_volume.npy to s3://test-bucket-sid/alex_neuroglancer_volumes/MD585/human_annotation/structure_volumes_10um/10N_L_vol

upload: ../../../../../media/alexn/BstemAtlasDataBackup/neuroglancer_volumes/MD585/human_annotation/structure_volumes_10um/SNC_L_volume.npy to s3://test-bucket-sid/alex_neuroglancer_volumes/MD585/human_annotation/structure_volumes_10um/SNC_L_volume.npy
upload: ../../../../../media/alexn/BstemAtlasDataBackup/neuroglancer_volumes/MD585/human_annotation/structure_volumes_10um/LRt_R_volume.npy to s3://test-bucket-sid/alex_neuroglancer_volumes/MD585/human_annotation/structure_volumes_10um/LRt_R_volume.npy
upload: ../../../../../media/alexn/BstemAtlasDataBackup/neuroglancer_volumes/MD585/human_annotation/structure_volumes_10um/SNR_L_volume.npy to s3://test-bucket-sid/alex_neuroglancer_volumes/MD585/human_annotation/structure_volumes_10um/SNR_L_volume.npy
upload: ../../../../../media/alexn/BstemAtlasDataBackup/neuroglancer_volumes/MD585/human_annotation/structure_volumes_10um/SC_volume.npy to s3://test-bucket-sid/alex_neuroglancer_volumes/MD585/human_annotation/structure_volumes_10um/SC_volum

upload: ../../../../../media/alexn/BstemAtlasDataBackup/neuroglancer_volumes/MD585/human_annotation/structure_volumes_50um/Amb_L_volume.npy to s3://test-bucket-sid/alex_neuroglancer_volumes/MD585/human_annotation/structure_volumes_50um/Amb_L_volume.npy
upload: ../../../../../media/alexn/BstemAtlasDataBackup/neuroglancer_volumes/MD585/human_annotation/structure_volumes_50um/7n_R_volume.npy to s3://test-bucket-sid/alex_neuroglancer_volumes/MD585/human_annotation/structure_volumes_50um/7n_R_volume.npy
upload: ../../../../../media/alexn/BstemAtlasDataBackup/neuroglancer_volumes/MD585/human_annotation/structure_volumes_50um/AP_volume.npy to s3://test-bucket-sid/alex_neuroglancer_volumes/MD585/human_annotation/structure_volumes_50um/AP_volume.npy
upload: ../../../../../media/alexn/BstemAtlasDataBackup/neuroglancer_volumes/MD585/human_annotation/structure_volumes_50um/Amb_R_volume.npy to s3://test-bucket-sid/alex_neuroglancer_volumes/MD585/human_annotation/structure_volumes_50um/Amb_R_volume.

upload: ../../../../../media/alexn/BstemAtlasDataBackup/neuroglancer_volumes/MD585/human_annotation/structure_volumes_50um/Sp5I_R_volume.npy to s3://test-bucket-sid/alex_neuroglancer_volumes/MD585/human_annotation/structure_volumes_50um/Sp5I_R_volume.npy
upload: ../../../../../media/alexn/BstemAtlasDataBackup/neuroglancer_volumes/MD585/human_annotation/structure_volumes_50um/VCA_R_volume.npy to s3://test-bucket-sid/alex_neuroglancer_volumes/MD585/human_annotation/structure_volumes_50um/VCA_R_volume.npy
upload: ../../../../../media/alexn/BstemAtlasDataBackup/neuroglancer_volumes/MD585/human_annotation/structure_volumes_50um/VLL_L_volume.npy to s3://test-bucket-sid/alex_neuroglancer_volumes/MD585/human_annotation/structure_volumes_50um/VLL_L_volume.npy
upload: ../../../../../media/alexn/BstemAtlasDataBackup/neuroglancer_volumes/MD585/human_annotation/structure_volumes_50um/VCP_R_volume.npy to s3://test-bucket-sid/alex_neuroglancer_volumes/MD585/human_annotation/structure_volumes_50um/VCP

upload: ../../../../../media/alexn/BstemAtlasDataBackup/neuroglancer_volumes/MD585/human_annotation/structure_volumes_5um/Pn_R_volume.npy to s3://test-bucket-sid/alex_neuroglancer_volumes/MD585/human_annotation/structure_volumes_5um/Pn_R_volume.npy
upload: ../../../../../media/alexn/BstemAtlasDataBackup/neuroglancer_volumes/MD585/human_annotation/structure_volumes_5um/RMC_R_volume.npy to s3://test-bucket-sid/alex_neuroglancer_volumes/MD585/human_annotation/structure_volumes_5um/RMC_R_volume.npy
upload: ../../../../../media/alexn/BstemAtlasDataBackup/neuroglancer_volumes/MD585/human_annotation/structure_volumes_5um/RMC_L_volume.npy to s3://test-bucket-sid/alex_neuroglancer_volumes/MD585/human_annotation/structure_volumes_5um/RMC_L_volume.npy
upload: ../../../../../media/alexn/BstemAtlasDataBackup/neuroglancer_volumes/MD585/human_annotation/structure_volumes_5um/RtTg_volume.npy to s3://test-bucket-sid/alex_neuroglancer_volumes/MD585/human_annotation/structure_volumes_5um/RtTg_volume.npy


...

upload: ../../../../../media/alexn/BstemAtlasDataBackup/neuroglancer_volumes/MD594/human_annotation/structure_volumes_10um/5N_L_volume.npy to s3://test-bucket-sid/alex_neuroglancer_volumes/MD594/human_annotation/structure_volumes_10um/5N_L_volume.npy
upload: ../../../../../media/alexn/BstemAtlasDataBackup/neuroglancer_volumes/MD594/human_annotation/structure_volumes_10um/4N_L_volume.npy to s3://test-bucket-sid/alex_neuroglancer_volumes/MD594/human_annotation/structure_volumes_10um/4N_L_volume.npy
upload: ../../../../../media/alexn/BstemAtlasDataBackup/neuroglancer_volumes/MD594/human_annotation/structure_volumes_10um/6N_R_volume.npy to s3://test-bucket-sid/alex_neuroglancer_volumes/MD594/human_annotation/structure_volumes_10um/6N_R_volume.npy
upload: ../../../../../media/alexn/BstemAtlasDataBackup/neuroglancer_volumes/MD594/human_annotation/structure_volumes_10um/7N_L_volume.npy to s3://test-bucket-sid/alex_neuroglancer_volumes/MD594/human_annotation/structure_volumes_10um/7N_L_volume.

upload: ../../../../../media/alexn/BstemAtlasDataBackup/neuroglancer_volumes/MD594/human_annotation/structure_volumes_10um/Sp5C_L_volume.npy to s3://test-bucket-sid/alex_neuroglancer_volumes/MD594/human_annotation/structure_volumes_10um/Sp5C_L_volume.npy
upload: ../../../../../media/alexn/BstemAtlasDataBackup/neuroglancer_volumes/MD594/human_annotation/structure_volumes_10um/Sp5O_L_volume.npy to s3://test-bucket-sid/alex_neuroglancer_volumes/MD594/human_annotation/structure_volumes_10um/Sp5O_L_volume.npy
upload: ../../../../../media/alexn/BstemAtlasDataBackup/neuroglancer_volumes/MD594/human_annotation/structure_volumes_10um/Tz_L_volume.npy to s3://test-bucket-sid/alex_neuroglancer_volumes/MD594/human_annotation/structure_volumes_10um/Tz_L_volume.npy
upload: ../../../../../media/alexn/BstemAtlasDataBackup/neuroglancer_volumes/MD594/human_annotation/structure_volumes_10um/Sp5C_R_volume.npy to s3://test-bucket-sid/alex_neuroglancer_volumes/MD594/human_annotation/structure_volumes_10um/Sp

upload: ../../../../../media/alexn/BstemAtlasDataBackup/neuroglancer_volumes/MD594/human_annotation/structure_volumes_15um/PBG_L_volume.npy to s3://test-bucket-sid/alex_neuroglancer_volumes/MD594/human_annotation/structure_volumes_15um/PBG_L_volume.npy
upload: ../../../../../media/alexn/BstemAtlasDataBackup/neuroglancer_volumes/MD594/human_annotation/structure_volumes_15um/Amb_R_volume.npy to s3://test-bucket-sid/alex_neuroglancer_volumes/MD594/human_annotation/structure_volumes_15um/Amb_R_volume.npy
upload: ../../../../../media/alexn/BstemAtlasDataBackup/neuroglancer_volumes/MD594/human_annotation/structure_volumes_15um/LC_R_volume.npy to s3://test-bucket-sid/alex_neuroglancer_volumes/MD594/human_annotation/structure_volumes_15um/LC_R_volume.npy
upload: ../../../../../media/alexn/BstemAtlasDataBackup/neuroglancer_volumes/MD594/human_annotation/structure_volumes_15um/DC_R_volume.npy to s3://test-bucket-sid/alex_neuroglancer_volumes/MD594/human_annotation/structure_volumes_15um/DC_R_vol

upload: ../../../../../media/alexn/BstemAtlasDataBackup/neuroglancer_volumes/MD594/human_annotation/structure_volumes_5um/3N_R_volume.npy to s3://test-bucket-sid/alex_neuroglancer_volumes/MD594/human_annotation/structure_volumes_5um/3N_R_volume.npy
upload: ../../../../../media/alexn/BstemAtlasDataBackup/neuroglancer_volumes/MD594/human_annotation/structure_volumes_5um/10N_L_volume.npy to s3://test-bucket-sid/alex_neuroglancer_volumes/MD594/human_annotation/structure_volumes_5um/10N_L_volume.npy
upload: ../../../../../media/alexn/BstemAtlasDataBackup/neuroglancer_volumes/MD594/human_annotation/structure_volumes_5um/10N_R_volume.npy to s3://test-bucket-sid/alex_neuroglancer_volumes/MD594/human_annotation/structure_volumes_5um/10N_R_volume.npy
upload: ../../../../../media/alexn/BstemAtlasDataBackup/neuroglancer_volumes/MD594/human_annotation/structure_volumes_5um/12N_volume.npy to s3://test-bucket-sid/alex_neuroglancer_volumes/MD594/human_annotation/structure_volumes_5um/12N_volume.npy
up

upload: ../../../../../media/alexn/BstemAtlasDataBackup/neuroglancer_volumes/MD594/human_annotation/structure_volumes_5um/SNR_L_volume.npy to s3://test-bucket-sid/alex_neuroglancer_volumes/MD594/human_annotation/structure_volumes_5um/SNR_L_volume.npy
upload: ../../../../../media/alexn/BstemAtlasDataBackup/neuroglancer_volumes/MD594/human_annotation/structure_volumes_5um/Sp5I_L_volume.npy to s3://test-bucket-sid/alex_neuroglancer_volumes/MD594/human_annotation/structure_volumes_5um/Sp5I_L_volume.npy
upload: ../../../../../media/alexn/BstemAtlasDataBackup/neuroglancer_volumes/MD594/human_annotation/structure_volumes_5um/SNR_R_volume.npy to s3://test-bucket-sid/alex_neuroglancer_volumes/MD594/human_annotation/structure_volumes_5um/SNR_R_volume.npy
upload: ../../../../../media/alexn/BstemAtlasDataBackup/neuroglancer_volumes/MD594/human_annotation/structure_volumes_5um/Sp5C_L_volume.npy to s3://test-bucket-sid/alex_neuroglancer_volumes/MD594/human_annotation/structure_volumes_5um/Sp5C_L_vol

In [10]:
os.environ['ROOT_DIR']

'/media/alexn/BstemAtlasDataBackup/ucsd_brain/'

# End Neuroglancer Code

In [14]:
# Resets the viewer
with viewer.txn() as s:
    # Resets X/Y/Z plane orientation
    s.navigation.pose.orientation = [0,0,0,1]
    # Zooms out 
    s.navigation.zoomFactor = 10000
    
    # Resets 3D Viewer Orientation
    s.perspectiveOrientation = [0,0,0,1]
    # Zooms out
    s.perspectiveZoom = 75000
    
    # Not necessary, just restates the voxel sizes of the image
    s.navigation.pose.position.voxelSize = [460,460,20000]
    # Sets Viewer's center location
    s.navigation.pose.position.voxelCoordinates = [8192,6000,134]

In [ ]:
# wholeslice_to_brainstem = -from_padded_to_wholeslice, from_padded_to_brainstem

#from_padded_to_wholeslice
rostral_limit = 50
caudal_limit = 1188
dorsal_limit = 21
ventral_limit = 738

#from_padded_to_brainstem
rostral_limit = 521
caudal_limit = 1057
dorsal_limit = 128
ventral_limit = 465

In [ ]:
fp_volumes = '/home/alexn/Desktop/neuroglancer_binary_volumes/volumes_'+xy_ng_resolution_um+'um/'
np.save( fp_volumes+structure+'_volume.npy',structure_volume)